In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
data  = pd.read_excel("../predict_data_test.xlsx",engine='openpyxl')

In [6]:
corr_data = data.corr()

l = []


for k1 in corr_data:
    
    for k2 in corr_data:
        value = corr_data[k1][k2]
        if k1 != k2:
            if value > 0.97:
                for i in l:
                    if k1 == i[1] and k2 == i[0]:
                        
                        break
                else:
                   
                    l.append((k1,k2,value))
 
l

[('qty_slash_url', 'qty_slash_directory', 0.9987353867586225),
 ('qty_equal_url', 'qty_equal_params', 0.9733632454761673),
 ('qty_comma_url', 'qty_comma_file', 1.0),
 ('qty_equal_directory', 'qty_at_directory', 0.9999999999999986),
 ('qty_equal_directory', 'qty_and_directory', 0.9999999999999986),
 ('qty_at_directory', 'qty_and_directory', 1.0),
 ('qty_equal_params', 'qty_params', 0.9939871978690746)]

In [10]:
filtered_data = data.copy()

for i in l:
    filtered_data.drop(labels=i[0],axis=1,inplace=True,errors='ignore') 
    
print(filtered_data.columns) 

Index(['Unnamed: 0', 'qty_dot_url', 'qty_hyphen_url', 'qty_underline_url',
       'qty_questionmark_url', 'qty_at_url', 'qty_and_url',
       'qty_exclamation_url', 'qty_space_url', 'qty_tilde_url',
       ...
       'qty_nameservers', 'qty_mx_servers', 'ttl_hostname',
       'tls_ssl_certificate', 'qty_redirects', 'url_google_index',
       'domain_google_index', 'url_shortened', 'url', 'Labels'],
      dtype='object', length=108)


In [11]:
filtered_data['Labels'].value_counts()

0    215
1    133
Name: Labels, dtype: int64

In [14]:
 pip install -U imbalanced-learn


  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
from imblearn.over_sampling import SMOTE

In [18]:
X = filtered_data.drop(columns=["Labels","url"])
y = filtered_data['Labels']
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

In [35]:
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score,cross_validate
scoring = ['precision_macro', 'recall_macro']


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=42)


In [37]:
from sklearn.ensemble import RandomForestClassifier


In [39]:
clf = RandomForestClassifier(max_depth=2,n_estimators=200, random_state=0)
pipeline = make_pipeline(PowerTransformer(), clf)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
cross_validate(pipeline, X_resampled, y_resampled, scoring=scoring,cv = 5)

d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


              precision    recall  f1-score   support

           0       0.76      0.83      0.79        53
           1       0.82      0.75      0.78        55

    accuracy                           0.79       108
   macro avg       0.79      0.79      0.79       108
weighted avg       0.79      0.79      0.79       108



d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


{'fit_time': array([0.23174191, 0.22699213, 0.22656059, 0.21899939, 0.22500014]),
 'score_time': array([0.01100063, 0.00999951, 0.01099944, 0.01000023, 0.00999975]),
 'test_precision_macro': array([0.72885154, 0.82521008, 0.73830049, 0.87229437, 0.79646994]),
 'test_recall_macro': array([0.72093023, 0.81395349, 0.70930233, 0.87209302, 0.79069767])}

In [38]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)
pipeline = make_pipeline(PowerTransformer(), clf)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

cross_validate(pipeline, X_resampled, y_resampled, scoring=scoring,cv = 5)

d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


              precision    recall  f1-score   support

           0       0.84      0.77      0.80        53
           1       0.80      0.85      0.82        55

    accuracy                           0.81       108
   macro avg       0.82      0.81      0.81       108
weighted avg       0.82      0.81      0.81       108



d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


{'fit_time': array([0.13916421, 0.13200116, 0.12745404, 0.12995005, 0.12790728]),
 'score_time': array([0.00499701, 0.00400376, 0.00304079, 0.00308681, 0.0039916 ]),
 'test_precision_macro': array([0.79646994, 0.83190395, 0.81773399, 0.85054348, 0.85833333]),
 'test_recall_macro': array([0.79069767, 0.81395349, 0.77906977, 0.84883721, 0.8255814 ])}

In [28]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
pipeline = make_pipeline(PowerTransformer(), clf)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.94      0.70        53
           1       0.84      0.29      0.43        55

    accuracy                           0.61       108
   macro avg       0.70      0.62      0.57       108
weighted avg       0.70      0.61      0.57       108



d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [40]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
pipeline = make_pipeline(PowerTransformer(), clf)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

cross_validate(pipeline, X_resampled, y_resampled, scoring=scoring,cv = 5)


d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


              precision    recall  f1-score   support

           0       0.73      0.68      0.71        53
           1       0.71      0.76      0.74        55

    accuracy                           0.72       108
   macro avg       0.72      0.72      0.72       108
weighted avg       0.72      0.72      0.72       108



d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


{'fit_time': array([0.11261606, 0.11362338, 0.11484647, 0.11264157, 0.12060285]),
 'score_time': array([0.00400567, 0.00400019, 0.00300002, 0.00399828, 0.00400758]),
 'test_precision_macro': array([0.73958944, 0.85361842, 0.77703081, 0.84015276, 0.82521008]),
 'test_recall_macro': array([0.72093023, 0.84883721, 0.76744186, 0.8372093 , 0.81395349])}

In [41]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100, random_state=0)
pipeline = make_pipeline(PowerTransformer(), clf)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

cross_validate(pipeline, X_resampled, y_resampled, scoring=scoring,cv = 5)

d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


              precision    recall  f1-score   support

           0       0.83      0.83      0.83        53
           1       0.84      0.84      0.84        55

    accuracy                           0.83       108
   macro avg       0.83      0.83      0.83       108
weighted avg       0.83      0.83      0.83       108



d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


{'fit_time': array([0.19988322, 0.20138121, 0.19993329, 0.20275569, 0.19640231]),
 'score_time': array([0.00999951, 0.00903416, 0.00900173, 0.00932121, 0.00901127]),
 'test_precision_macro': array([0.72885154, 0.78289474, 0.80731847, 0.92065217, 0.89090909]),
 'test_recall_macro': array([0.72093023, 0.77906977, 0.79069767, 0.91860465, 0.86046512])}

In [42]:
from xgboost import XGBClassifier
clf = XGBClassifier()
pipeline = make_pipeline(PowerTransformer(), clf)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
cross_validate(pipeline, X_resampled, y_resampled, scoring=scoring,cv = 5)

d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


              precision    recall  f1-score   support

           0       0.83      0.81      0.82        53
           1       0.82      0.84      0.83        55

    accuracy                           0.82       108
   macro avg       0.82      0.82      0.82       108
weighted avg       0.82      0.82      0.82       108



d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
d:\FSDS\DS internship\phising_project\venv\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


{'fit_time': array([0.23431158, 0.23587823, 0.23009968, 0.236063  , 0.22737336]),
 'score_time': array([0.00800109, 0.00699949, 0.00900006, 0.00799942, 0.00769377]),
 'test_precision_macro': array([0.84015276, 0.84902597, 0.85648942, 0.95447154, 0.92156863]),
 'test_recall_macro': array([0.8372093 , 0.84883721, 0.8372093 , 0.95348837, 0.90697674])}